In [35]:
from flask import Flask, render_template, request, redirect, url_for, flash, session
from flask_sqlalchemy import SQLAlchemy
from werkzeug.security import generate_password_hash, check_password_hash


In [ ]:
app = Flask(__name__)
app.config['SECRET_KEY'] = "JPMorgan123!"
app.config['SQLALCHEMY_DATABASE_URI'] = r'sqlite:///C:\Users\yaguc\Downloads\JP Morgan Documents\Week 3\Day 5\Blog Platform Project\database\Blog_Platform_Database.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)

In [41]:
class User(db.Model):
    user_id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(50), unique=True, nullable=False)
    password_hash = db.Column(db.String(255), nullable=False)
    email = db.Column(db.String(100), unique=True, nullable=False)
    role = db.Column(db.Enum('Admin', 'Author', 'Reader'), default='Reader')
    created_at = db.Column(db.DateTime, default=db.func.current_timestamp())

In [43]:
class Post(db.Model):
    post_id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(255), nullable=False)
    content = db.Column(db.Text, nullable=False)
    author_id = db.Column(db.Integer, db.ForeignKey('user.user_id'))
    created_at = db.Column(db.DateTime, default=db.func.current_timestamp())
    updated_at = db.Column(db.DateTime, default=db.func.current_timestamp(), onupdate=db.func.current_timestamp())
    category = db.Column(db.String(100))
    tags = db.Column(db.Text)

In [45]:
class Comment(db.Model):
    comment_id = db.Column(db.Integer, primary_key=True)
    post_id = db.Column(db.Integer, db.ForeignKey('post.post_id'))
    user_id = db.Column(db.Integer, db.ForeignKey('user.user_id'))
    comment = db.Column(db.Text, nullable=False)
    created_at = db.Column(db.DateTime, default=db.func.current_timestamp())

In [47]:
class Tag(db.Model):
    tag_id = db.Column(db.Integer, primary_key=True)
    tag_name = db.Column(db.String(100), unique=True, nullable=False)

In [49]:
class PostTag(db.Model):
    post_tag_id = db.Column(db.Integer, primary_key=True)
    post_id = db.Column(db.Integer, db.ForeignKey('post.post_id'))
    tag_id = db.Column(db.Integer, db.ForeignKey('tag.tag_id'))

In [51]:
with app.app_context():
    db.create_all()

In [53]:
@app.route('/')
def index():
    posts = Post.query.all()
    return render_template('frontend/index.html', posts=posts)

In [55]:
@app.route('/post/<int:post_id>')
def post_detail(post_id):
    post = Post.query.get_or_404(post_id)
    comments = Comment.query.filter_by(post_id=post_id).all()
    return render_template('frontend/post_detail.html', post=post, comments=comments)

In [57]:
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']
        user = User.query.filter_by(username=username).first()
        if user and check_password_hash(user.password_hash, password):
            session['user_id'] = user.user_id
            flash('Login successful!', 'success')
            return redirect('/')
        else:
            flash('Login failed! Check your credentials', 'danger')
    return render_template('frontend/login.html')

In [59]:
@app.route('/register', methods=['GET', 'POST'])
def register():
    if request.method == 'POST':
        username = request.form['username']
        email = request.form['email']
        password = request.form['password']
        hashed_password = generate_password_hash(password, method='sha256')
        new_user = User(username=username, password_hash=hashed_password, email=email)
        db.session.add(new_user)
        db.session.commit()
        flash('Registration successful! You can now log in.', 'success')
        return redirect(url_for('index'))
    return render_template('frontend/register.html')

In [61]:
def add_comment(post_id):
    if 'user_id' in session:
        comment_text = request.form['comment']
        new_comment = Comment(post_id=post_id, user_id=session['user_id'], comment=comment_text)
        db.session.add(new_comment)
        db.session.commit()
        flash('Comment added successfully!', 'success')
    else:
        flash('You need to be logged in to comment', 'danger')
    return redirect(url_for('post_detail', post_id=post_id))

In [69]:
if __name__ == '__main__':
    app.run(host='127.0.0.1', port=5000, debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1